In [296]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_style('whitegrid')

In [297]:
df_test_features = pd.read_csv('../test_features.csv')

In [298]:
num_test_data = df_test_features.iloc[:,9:]
num_test_data['loudness'] = df_test_features['loudness']
num_test_data['tempo'] = df_test_features['tempo']
num_test_data['duration'] = df_test_features['duration']
cat_test_data = df_test_features[['time_signature', 'key', 'mode']]

In [299]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [300]:
cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
cat_test_data['mode_cat'] = le.fit_transform(cat_test_data['mode'])

C:\Users\alex0\AppData\Local\Temp/ipykernel_26472/719706249.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['time_signature_cat'] = le.fit_transform(cat_test_data['time_signature'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_26472/719706249.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_test_data['key_cat'] = le.fit_transform(cat_test_data['key'])
C:\Users\alex0\AppData\Local\Temp/ipykernel_26472/719706249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [301]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [302]:
num_test = scaler.fit_transform(num_test_data)

In [303]:
import ktrain

In [304]:
predictor = ktrain.load_predictor('../text_only')

# Pre-process string with ktrain

In [305]:
def str_remove_comma(string):
    str = string.split(", ")
    return str

def list_to_string(list):
    str = " "
    return str.join(list)

In [306]:
# remove stopwords
import nltk
stopwords = nltk.corpus.stopwords.words('english')

In [307]:
x_test2 = df_test_features['tags'].apply(lambda x : list_to_string(str_remove_comma(x)))
x_test2 = x_test2.apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
x_test2 = np.array(x_test2)

In [308]:
x_text_input = predictor.preproc.preprocess_test(x_test2)

428 test sequences
test sequence lengths:
	mean : 54
	95percentile : 109
	99percentile : 150
x_test shape: (428,150)


# Make a prediction

In [309]:
test_dataset = [x_text_input[0]]

In [310]:
prediction = predictor.model.predict(test_dataset)

In [311]:
predict_labels = [ label.argmax() for label in prediction]
df = pd.DataFrame(predict_labels, columns=['genre_pred'])
classes_mapping = {
    0: 'classic pop and rock',
    1: 'dance and electronica',
    2: 'folk',
    3: 'jazz and blues',
    4: 'metal',
    5: 'pop',
    6: 'punk',
    7: 'soul and reggae',
}
df['genre_pred'] = df['genre_pred'].map(classes_mapping)
df['trackID'] = df_test_features['trackID']

In [312]:
correct_label = pd.read_csv('../test_labels.csv')
df_merged = pd.merge(df, correct_label, on='trackID')

In [313]:
from sklearn.metrics import classification_report
print(classification_report(df_merged['genre'], df_merged['genre_pred']))

                       precision    recall  f1-score   support

 classic pop and rock       0.23      0.17      0.20        64
dance and electronica       0.16      0.09      0.12        64
                 folk       0.46      0.42      0.44        73
       jazz and blues       0.32      0.36      0.34        36
                metal       0.59      0.48      0.53        42
                  pop       0.83      1.00      0.91        44
                 punk       0.36      0.40      0.38        40
      soul and reggae       0.38      0.62      0.47        65

             accuracy                           0.42       428
            macro avg       0.42      0.44      0.42       428
         weighted avg       0.40      0.42      0.40       428

